<a href="https://colab.research.google.com/github/chltjs1921/2024_modeling_practice/blob/main/Graph_reconstruction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

화학에서의 graph 사용  
* 화학식을 graph로 전처리하는 것  
* 주로 SMILES>mol>특성추출의 과정을 거친다  
* 추출된 특성은 원자의 경우 node로, 결합의 경우 edge로 간다  
* 특성 중 숫자가 아닌 것들(예: C, SP3 등) 은 encoding 과정을 거쳐서 숫자로 변환된 후 이동한다. encoding에는 one-hot 및 label 사용 https://coduking.tistory.com/entry/%EC%9B%90-%ED%95%AB-%EC%9D%B8%EC%BD%94%EB%94%A9-pdgetdummies-vs-OneHotEncoder

dataset의 prop를 추출해서 그것으로 list나 set를 만들고 one-hot?  
그래프는 dgl이냐 torch geometric이냐  
https://pytorch-geometric.readthedocs.io/en/latest/modules/utils.html#torch_geometric.utils.from_smiles    
https://pytorch-geometric.readthedocs.io/en/latest/_modules/torch_geometric/utils/smiles.html#from_smiles  
Converts a SMILES string to a torch_geometric.data.Data instance.  

1, 0, 0, ... 이 아닌 1, 2, 3, ... 주로 사용

Parameters
:
* smiles (str) – The SMILES string.

* with_hydrogen (bool, optional) – If set to True, will store hydrogens in the molecule graph. (default: False)

* kekulize (bool, optional) – If set to True, converts aromatic bonds to single/double bonds. (default: False)  
  
해당 함수를 그대로 쓸 것인가 customize한 함수(예:1, 0, 0, ... )를 만들 것인가  

0을 최소화하기 위함인가?

In [1]:
import torch

print(torch.__version__)
print(torch.version.cuda)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

2.4.0+cu121
12.1
cuda


In [2]:
!pip install rdkit
!pip install torch_geometric

# Optional dependencies: 버전 꼭 맞추기!!!!!
!pip install pyg_lib torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-2.4.0+cu121.html

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.1/33.1 MB 28.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 19.5 MB/s eta 0:00:00
Looking in links: https://data.pyg.org/whl/torch-2.4.0+cu121.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 111.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 90.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 89.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 986.2/986.2 kB 49.7 MB/s eta 0:00:00


In [ ]:
# Source code for torch_geometric.utils.smiles
# Ref: https://pytorch-geometric.readthedocs.io/en/latest/_modules/torch_geometric/utils/smiles.html#from_smiles

from typing import Any, Dict, List

import torch

import torch_geometric

x_map: Dict[str, List[Any]] = {
    'atomic_num':
    list(range(0, 119)),
    'chirality': [
        'CHI_UNSPECIFIED',
        'CHI_TETRAHEDRAL_CW',
        'CHI_TETRAHEDRAL_CCW',
        'CHI_OTHER',
        'CHI_TETRAHEDRAL',
        'CHI_ALLENE',
        'CHI_SQUAREPLANAR',
        'CHI_TRIGONALBIPYRAMIDAL',
        'CHI_OCTAHEDRAL',
    ],
    'degree':
    list(range(0, 11)),
    'formal_charge':
    list(range(-5, 7)),
    'num_hs':
    list(range(0, 9)),
    'num_radical_electrons':
    list(range(0, 5)),
    'hybridization': [
        'UNSPECIFIED',
        'S',
        'SP',
        'SP2',
        'SP3',
        'SP3D',
        'SP3D2',
        'OTHER',
    ],
    'is_aromatic': [False, True],
    'is_in_ring': [False, True],
}

e_map: Dict[str, List[Any]] = {
    'bond_type': [
        'UNSPECIFIED',
        'SINGLE',
        'DOUBLE',
        'TRIPLE',
        'QUADRUPLE',
        'QUINTUPLE',
        'HEXTUPLE',
        'ONEANDAHALF',
        'TWOANDAHALF',
        'THREEANDAHALF',
        'FOURANDAHALF',
        'FIVEANDAHALF',
        'AROMATIC',
        'IONIC',
        'HYDROGEN',
        'THREECENTER',
        'DATIVEONE',
        'DATIVE',
        'DATIVEL',
        'DATIVER',
        'OTHER',
        'ZERO',
    ],
    'stereo': [
        'STEREONONE',
        'STEREOANY',
        'STEREOZ',
        'STEREOE',
        'STEREOCIS',
        'STEREOTRANS',
    ],
    'is_conjugated': [False, True],
}


[docs]def from_rdmol(mol: Any) -> 'torch_geometric.data.Data':
    r"""Converts a :class:`rdkit.Chem.Mol` instance to a
    :class:`torch_geometric.data.Data` instance.

    Args:
        mol (rdkit.Chem.Mol): The :class:`rdkit` molecule.
    """
    from rdkit import Chem

    from torch_geometric.data import Data

    assert isinstance(mol, Chem.Mol)

    xs: List[List[int]] = []
    for atom in mol.GetAtoms():  # type: ignore
        row: List[int] = []
        row.append(x_map['atomic_num'].index(atom.GetAtomicNum()))
        row.append(x_map['chirality'].index(str(atom.GetChiralTag())))
        row.append(x_map['degree'].index(atom.GetTotalDegree()))
        row.append(x_map['formal_charge'].index(atom.GetFormalCharge()))
        row.append(x_map['num_hs'].index(atom.GetTotalNumHs()))
        row.append(x_map['num_radical_electrons'].index(
            atom.GetNumRadicalElectrons()))
        row.append(x_map['hybridization'].index(str(atom.GetHybridization())))
        row.append(x_map['is_aromatic'].index(atom.GetIsAromatic()))
        row.append(x_map['is_in_ring'].index(atom.IsInRing()))
        xs.append(row)

    x = torch.tensor(xs, dtype=torch.long).view(-1, 9)

    edge_indices, edge_attrs = [], []
    for bond in mol.GetBonds():  # type: ignore
        i = bond.GetBeginAtomIdx()
        j = bond.GetEndAtomIdx()

        e = []
        e.append(e_map['bond_type'].index(str(bond.GetBondType())))
        e.append(e_map['stereo'].index(str(bond.GetStereo())))
        e.append(e_map['is_conjugated'].index(bond.GetIsConjugated()))

        edge_indices += [[i, j], [j, i]]
        edge_attrs += [e, e]

    edge_index = torch.tensor(edge_indices)
    edge_index = edge_index.t().to(torch.long).view(2, -1)
    edge_attr = torch.tensor(edge_attrs, dtype=torch.long).view(-1, 3)

    if edge_index.numel() > 0:  # Sort indices.
        perm = (edge_index[0] * x.size(0) + edge_index[1]).argsort()
        edge_index, edge_attr = edge_index[:, perm], edge_attr[perm]

    return Data(x=x, edge_index=edge_index, edge_attr=edge_attr)


[docs]def from_smiles(
    smiles: str,
    with_hydrogen: bool = False,
    kekulize: bool = False,
) -> 'torch_geometric.data.Data':
    r"""Converts a SMILES string to a :class:`torch_geometric.data.Data`
    instance.

    Args:
        smiles (str): The SMILES string.
        with_hydrogen (bool, optional): If set to :obj:`True`, will store
            hydrogens in the molecule graph. (default: :obj:`False`)
        kekulize (bool, optional): If set to :obj:`True`, converts aromatic
            bonds to single/double bonds. (default: :obj:`False`)
    """
    from rdkit import Chem, RDLogger

    RDLogger.DisableLog('rdApp.*')  # type: ignore

    mol = Chem.MolFromSmiles(smiles)

    if mol is None:
        mol = Chem.MolFromSmiles('')
    if with_hydrogen:
        mol = Chem.AddHs(mol)
    if kekulize:
        Chem.Kekulize(mol)

    data = from_rdmol(mol)
    data.smiles = smiles
    return data


[docs]def to_rdmol(
    data: 'torch_geometric.data.Data',
    kekulize: bool = False,
) -> Any:
    """Converts a :class:`torch_geometric.data.Data` instance to a
    :class:`rdkit.Chem.Mol` instance.

    Args:
        data (torch_geometric.data.Data): The molecular graph data.
        kekulize (bool, optional): If set to :obj:`True`, converts aromatic
            bonds to single/double bonds. (default: :obj:`False`)
    """
    from rdkit import Chem

    mol = Chem.RWMol()

    assert data.x is not None
    assert data.num_nodes is not None
    assert data.edge_index is not None
    assert data.edge_attr is not None
    for i in range(data.num_nodes):
        atom = Chem.Atom(int(data.x[i, 0]))
        atom.SetChiralTag(Chem.rdchem.ChiralType.values[int(data.x[i, 1])])
        atom.SetFormalCharge(x_map['formal_charge'][int(data.x[i, 3])])
        atom.SetNumExplicitHs(x_map['num_hs'][int(data.x[i, 4])])
        atom.SetNumRadicalElectrons(x_map['num_radical_electrons'][int(
            data.x[i, 5])])
        atom.SetHybridization(Chem.rdchem.HybridizationType.values[int(
            data.x[i, 6])])
        atom.SetIsAromatic(bool(data.x[i, 7]))
        mol.AddAtom(atom)

    edges = [tuple(i) for i in data.edge_index.t().tolist()]
    visited = set()

    for i in range(len(edges)):
        src, dst = edges[i]
        if tuple(sorted(edges[i])) in visited:
            continue

        bond_type = Chem.BondType.values[int(data.edge_attr[i, 0])]
        mol.AddBond(src, dst, bond_type)

        # Set stereochemistry:
        stereo = Chem.rdchem.BondStereo.values[int(data.edge_attr[i, 1])]
        if stereo != Chem.rdchem.BondStereo.STEREONONE:
            db = mol.GetBondBetweenAtoms(src, dst)
            db.SetStereoAtoms(dst, src)
            db.SetStereo(stereo)

        # Set conjugation:
        is_conjugated = bool(data.edge_attr[i, 2])
        mol.GetBondBetweenAtoms(src, dst).SetIsConjugated(is_conjugated)

        visited.add(tuple(sorted(edges[i])))

    mol = mol.GetMol()

    if kekulize:
        Chem.Kekulize(mol)

    Chem.SanitizeMol(mol)
    Chem.AssignStereochemistry(mol)

    return mol


[docs]def to_smiles(
    data: 'torch_geometric.data.Data',
    kekulize: bool = False,
) -> str:
    """Converts a :class:`torch_geometric.data.Data` instance to a SMILES
    string.

    Args:
        data (torch_geometric.data.Data): The molecular graph.
        kekulize (bool, optional): If set to :obj:`True`, converts aromatic
            bonds to single/double bonds. (default: :obj:`False`)
    """
    from rdkit import Chem
    mol = to_rdmol(data, kekulize=kekulize)
    return Chem.MolToSmiles(mol, isomericSmiles=True)

나만의 customized 함수를 만들어본다면?  
source code만으로 충분치 아니하거나 과하다면

In [3]:
from rdkit import Chem
import torch_geometric
from torch_geometric.data import Data
import torch


# atom list: atom prop 넣기 RDKit으로 계산할 수 있는 Atom의 속성은 아래 링크에서 확인할 수 있다.
# https://www.rdkit.org/docs/source/rdkit.Chem.rdchem.html#rdkit.Chem.rdchem.Atom

# bond list: atom prop 넣기 RDKit으로 계산할 수 있는 Bond의 속성은 아래 링크에서 확인할 수 있다.
# https://www.rdkit.org/docs/source/rdkit.Chem.rdchem.html#rdkit.Chem.rdchem.Bond

In [4]:
smi = 'CC'
mol = Chem.MolFromSmiles(smi)
graph = torch_geometric.utils.smiles.from_smiles(smi)

In [5]:
graph

Data(x=[2, 9], edge_index=[2, 2], edge_attr=[2, 3], smiles='CC')

graph에 y value 포함? 굳이 그럴 필요는 없음>>넣어보자

In [ ]:
type(graph)

torch_geometric.data.data.Data

일단 하나하나씩 custom?  
  
데이터셋에 따라 유동적으로 할 수 있을까? >> 이럴려면 데이터셋을 읽고 값들 (예 : 'hybridization': [
        'UNSPECIFIED',
        'S',
        'SP',
        'SP2',
        'SP3',
        'SP3D',
        'SP3D2',
        'OTHER',
    ])을 추출 후 set을 만들고 dict 같은 거를 재구성해야 할 것. dataset의 해당 값들을 추출하려면?

graph 및 layer는 어디까지 custom?  
layer custom은 논문이나 ref code 기반으로 해야 할 것

이미 만들어진 그래프에 속성을 추가해보기

In [6]:
graph.y = torch.tensor([0])

In [ ]:
torch.tensor([0])

tensor([0])

In [ ]:
graph #요거 찍으면 size가 나옴. not value

Data(x=[2, 9], edge_index=[2, 2], edge_attr=[2, 3], smiles='CC', y=[3])

In [ ]:
graph.y

tensor([0, 0, 0])

In [ ]:
graph['y']

tensor([0, 0, 0])

그래프 y value는 길이? 그게 아님.   
y값을 확인하려면 graph['y'] 혹은 graph.y  
그래프 객체 찍기보단 이게 편함.  
모든 값을 확인하려면?

In [7]:
for key, item in graph:
    print(f'{key} : {item}')

x : tensor([[6, 0, 4, 5, 3, 0, 4, 0, 0],
        [6, 0, 4, 5, 3, 0, 4, 0, 0]])
edge_index : tensor([[0, 1],
        [1, 0]])
edge_attr : tensor([[1, 0, 0],
        [1, 0, 0]])
smiles : CC
y : tensor([0])


x: 원자 성질 크기는 원자 갯수 by 특성 갯수  
Node feature matrix with shape [num_nodes, num_node_features]   
edge_index:  
data.edge_index: Graph connectivity in COO format with shape [2, num_edges] and type torch.long  
edge_attr:  
data.edge_attr: Edge feature matrix with shape [num_edges, num_edge_features]
   
   smiles: SMILES  
y: endpoint

# PyTorch Geometric  
* PyTorch를 기반으로 구축된 오픈소스 라이브러리  
* 그래프 신경망 구성 및 데이터 핸들링 지원  
* 하나의 그래프를 torch_geometric.data.Data 인스턴스로 표현  
 *  Data.x : 그래프 정점에 대한 특성 행렬 → [num_nodes, num_node_features]
 *  Data.edge_index : 그래프 정점 간 연결 정보 → [2, num_edges]  
 *  Data.edge_attr : 연결 정보에 대한 특성 행렬 → [num_edges, num_edge_features]
 *  Data.y : 학습하고 싶은 대상 → [num_nodes, *] or [1, *]  
 *  Data.pos : 그래프 정점에 대한 위치 행렬 → [num_nodes, num_dimensions]

유동적 크기? padding!

In [38]:
#dataset 보기(이미 갖춰진 거)
from torch_geometric.datasets import QM9
from torch_geometric.loader import DataLoader

dataset = QM9(root='./')
loader = DataLoader(dataset, batch_size=32, shuffle=True)

for batch in loader:
    # print(batch)
    # print(batch.num_graphs)
    # print(batch.batch)
    # print(batch.ptr)
    # #print(batch.x.shape)
    # print(batch.edge_index.shape)
    # print(batch.edge_attr.shape)
    # print(batch.y.shape)
    # #print(batch.pos.shape)
    # break
    for key, item in batch:
        # if type(item) == torch.Tensor:
        #     print(f'{key} : {item.shape}')
        #print(f'{key} : {item}')
        print(key)
    break


/usr/local/lib/python3.10/dist-packages/torch_geometric/data/dataset.py:238: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  if osp.exists(f) and torch.load(f) != _repr(self.p

x
edge_index
edge_attr
y
pos
z
smiles
name
idx
batch
ptr


In [37]:
#dataset 보기
import numpy as np
import matplotlib.pylab as plt
import scipy.io
import pandas as pd

df = pd.read_csv('/content/raw/gdb9.sdf.csv')
df.head()

,mol_id,A,B,C,mu,alpha,homo,lumo,gap,r2,zpve,u0,u298,h298,g298,cv,u0_atom,u298_atom,h298_atom,g298_atom
0,gdb_1,157.71180,157.709970,157.706990,0.0000,13.21,-0.3877,0.1171,0.5048,35.3641,0.044749,-40.478930,-40.476062,-40.475117,-40.498597,6.469,-395.999595,-398.643290,-401.014647,-372.471772
1,gdb_2,293.60975,293.541110,191.393970,1.6256,9.46,-0.2570,0.0829,0.3399,26.1563,0.034358,-56.525887,-56.523026,-56.522082,-56.544961,6.316,-276.861363,-278.620271,-280.399259,-259.338802
2,gdb_3,799.58812,437.903860,282.945450,1.8511,6.31,-0.2928,0.0687,0.3615,19.0002,0.021375,-76.404702,-76.401867,-76.400922,-76.422349,6.002,-213.087624,-213.974294,-215.159658,-201.407171
3,gdb_4,0.00000,35.610036,35.610036,0.0000,16.28,-0.2845,0.0506,0.3351,59.5248,0.026841,-77.308427,-77.305527,-77.304583,-77.327429,8.574,-385.501997,-387.237686,-389.016047,-365.800724
4,gdb_5,0.00000,44.593883,44.593883,2.8937,12.99,-0.3604,0.0191,0.3796,48.7476,0.016601,-93.411888,-93.409370,-93.408425,-93.431246,6.278,-301.820534,-302.906752,-304.091489,-288.720028


In [62]:
for batch in loader:
    print(batch[0].smiles)
    print(batch[0].name)
    print(batch[0].x.shape)
    print(batch[0].edge_index.shape)
    print(batch[0].edge_attr.shape)
    print(batch[0].y.shape)
    break

#특성 19개
#앞의 3개만 따서 모델링 해보기
#df를 graph로 다시 만들어본다면?

[H]C#CC(=O)C(=O)C([H])([H])C#N
gdb_42356
torch.Size([12, 11])
torch.Size([2, 22])
torch.Size([22, 4])
torch.Size([1, 19])


In [40]:
df.shape #앞 빼고는 전부 target

(133885, 20)

In [41]:
from rdkit.Chem import Draw, PandasTools
from rdkit import Chem
import pandas as pd
from rdkit.Chem import Descriptors
from rdkit.ML.Descriptors import MoleculeDescriptors

df_from_sdf = PandasTools.LoadSDF('/content/raw/gdb9.sdf')
df_from_sdf.head()

,ID,ROMol
0,gdb_1,<rdkit.Chem.rdchem.Mol object at 0x790783b9e960>
1,gdb_2,<rdkit.Chem.rdchem.Mol object at 0x790783b9e9d0>
2,gdb_3,<rdkit.Chem.rdchem.Mol object at 0x790783b9ea40>
3,gdb_4,<rdkit.Chem.rdchem.Mol object at 0x790783b9eab0>
4,gdb_5,<rdkit.Chem.rdchem.Mol object at 0x790783b9eb20>


In [71]:
mol_example = df_from_sdf['ROMol'][42355]
smi_example = Chem.MolToSmiles(mol_example)
graph_example = torch_geometric.utils.smiles.from_smiles(smi_example, with_hydrogen=True, kekulize=True)
graph_example
#size 차이나는 이유? 수소와 custom(사전 dataset에는 feature들이 추가되어 있음. 원자 즉 edge 2개. 결합 즉 node는 하나 더 추가)
#근데 무슨 feature?
#모듈로 만든 기본 그래프로는 과연?

Data(x=[12, 9], edge_index=[2, 22], edge_attr=[22, 3], smiles='C#CC(=O)C(=O)CC#N')

In [75]:
#df 전부 그래프로
mol_from_df = df_from_sdf['ROMol']
smi_from_df = [Chem.MolToSmiles(mol) for mol in mol_from_df]
graph_from_df = [torch_geometric.utils.smiles.from_smiles(smi, with_hydrogen=True, kekulize=True) for smi in smi_from_df]
graph_from_df

[Data(x=[5, 9], edge_index=[2, 8], edge_attr=[8, 3], smiles='C'),
 Data(x=[4, 9], edge_index=[2, 6], edge_attr=[6, 3], smiles='N'),
 Data(x=[3, 9], edge_index=[2, 4], edge_attr=[4, 3], smiles='O'),
 Data(x=[4, 9], edge_index=[2, 6], edge_attr=[6, 3], smiles='C#C'),
 Data(x=[3, 9], edge_index=[2, 4], edge_attr=[4, 3], smiles='C#N'),
 Data(x=[4, 9], edge_index=[2, 6], edge_attr=[6, 3], smiles='C=O'),
 Data(x=[8, 9], edge_index=[2, 14], edge_attr=[14, 3], smiles='CC'),
 Data(x=[6, 9], edge_index=[2, 10], edge_attr=[10, 3], smiles='CO'),
 Data(x=[7, 9], edge_index=[2, 12], edge_attr=[12, 3], smiles='C#CC'),
 Data(x=[6, 9], edge_index=[2, 10], edge_attr=[10, 3], smiles='CC#N'),
 Data(x=[7, 9], edge_index=[2, 12], edge_attr=[12, 3], smiles='CC=O'),
 Data(x=[6, 9], edge_index=[2, 10], edge_attr=[10, 3], smiles='NC=O'),
 Data(x=[11, 9], edge_index=[2, 20], edge_attr=[20, 3], smiles='CCC'),
 Data(x=[9, 9], edge_index=[2, 16], edge_attr=[16, 3], smiles='CCO'),
 Data(x=[9, 9], edge_index=[2, 16],

In [76]:
type(graph_from_df)

list

In [77]:
from torch_geometric.loader import DataLoader

In [78]:
#split
train_set = graph_from_df[:int(len(graph_from_df)*0.8)]
test_set = graph_from_df[int(len(graph_from_df)*0.8):]

In [79]:
train_loader = DataLoader(train_set, batch_size=32, shuffle=True)
test_loader = DataLoader(test_set, batch_size=32, shuffle=False)

In [80]:
for batch in train_loader:
    print(batch)
    break

DataBatch(x=[570, 9], edge_index=[2, 1188], edge_attr=[1188, 3], smiles=[32], batch=[570], ptr=[33])


In [ ]:
#create model